In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [4]:
train = pd.read_csv('ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
train = train.dropna()

In [5]:
test = pd.read_csv('ta-en-misogyny-dev.csv',sep='\t', names = ["category", "text"])
test = test.dropna()

In [6]:
labels = train.category.unique()
labels

array(['None-of-the-above', 'Transphobic', 'Counter-speech', 'Misandry',
       'Homophobia', 'Hope-Speech', 'Xenophobia', 'Misogyny'],
      dtype=object)

In [7]:
train

,category,text
0,None-of-the-above,enaku unmaikum aluha wantu thirunangaigal thei...
1,Transphobic,SUPERSTAR VIJAY dai arivuketta polu ithu thapp...
2,None-of-the-above,Ugka smile cute a iruku😊😊...
3,None-of-the-above,Anna i am waiting na 🥰🥰🥰
4,None-of-the-above,Yanda tamilnadu la evvalavo pirachana iruku at...
...,...,...
5943,None-of-the-above,Noq day having no caste no religious certificate
5944,None-of-the-above,Mimicry escape aitaru bro. 😂
5945,None-of-the-above,Rajesh R u age u from
5946,Counter-speech,Bro unga videos nallarukku but don't do prank ...


In [8]:
test

,category,text
0,None-of-the-above,Black saree u r so hot
1,Counter-speech,Halo first ayunga kai thatti amount keatkyrang...
2,None-of-the-above,Hello...... manithana sollunga.......... prama...
3,Xenophobia,China kaaranai nalla kalaipan daily I am enjoy...
4,None-of-the-above,Onnota ponnu ipdi Panna vituviya
...,...,...
1483,Transphobic,Va d 9 usu vinoth
1484,None-of-the-above,Love panuradhaaaa soliyaeeee cover panura niiii😂
1485,None-of-the-above,Bro part 2 aippa varum
1486,Misandry,jii innum nariya thetunga andha potta gopiya


In [9]:
import string
def removepunctuation(text):
    clean="".join([i for i in text if i not in string.punctuation])
    return clean

import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def tolower(text):
  return text.lower()

label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

In [10]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga']) 
stopwords_list.extend(['enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha']) 

label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

def removestopwords(text):
  clean = " ".join([word for word in text.split() if word not in stopwords_list])
  return clean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
def preprocess(df):
  df['text']= df['text'].apply(lambda x:tolower(x))
  df['text']= df['text'].apply(lambda x:removepunctuation(x))
  df['text']= df['text'].apply(lambda x:deEmojify(x))
  df['text']= df['text'].apply(lambda x:removestopwords(x))
  df["category"] = df["category"].map(label)
  return df

In [12]:
train = preprocess(train)
train

,category,text
0,0,enaku unmaikum aluha wantu thirunangaigal thei...
1,1,superstar vijay arivuketta polu ithu thappu pa...
2,0,ugka smile cute
3,0,anna waiting 🥰🥰🥰
4,0,yanda tamilnadu evvalavo pirachana athalam vit...
...,...,...
5943,0,noq day caste religious certificate
5944,0,mimicry escape aitaru
5945,0,rajesh age
5946,2,videos nallarukku dont prank eena varavanga ep...


In [13]:
test = preprocess(test)

In [14]:
def tokenize(text):
  return text.split()


train['token']= train['text'].apply(lambda x:tokenize(x))
test['token']= test['text'].apply(lambda x:tokenize(x))

In [15]:
train

,category,text,token
0,0,enaku unmaikum aluha wantu thirunangaigal thei...,"[enaku, unmaikum, aluha, wantu, thirunangaigal..."
1,1,superstar vijay arivuketta polu ithu thappu pa...,"[superstar, vijay, arivuketta, polu, ithu, tha..."
2,0,ugka smile cute,"[ugka, smile, cute]"
3,0,anna waiting 🥰🥰🥰,"[anna, waiting, 🥰🥰🥰]"
4,0,yanda tamilnadu evvalavo pirachana athalam vit...,"[yanda, tamilnadu, evvalavo, pirachana, athala..."
...,...,...,...
5943,0,noq day caste religious certificate,"[noq, day, caste, religious, certificate]"
5944,0,mimicry escape aitaru,"[mimicry, escape, aitaru]"
5945,0,rajesh age,"[rajesh, age]"
5946,2,videos nallarukku dont prank eena varavanga ep...,"[videos, nallarukku, dont, prank, eena, varava..."


In [16]:
pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 5.5 MB/s 
     |████████████████████████████████| 3.5 MB 36.7 MB/s 
     |████████████████████████████████| 6.8 MB 51.8 MB/s 
     |████████████████████████████████| 1.2 MB 56.5 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 48.7 MB/s 
     |████████████████████████████████| 895 kB 67.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=9d1892d049bb3b461a5ac011b81dc07f269eeba74bff2103a766df85a277d41a
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [17]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/LaBSE')
embedding_train = embedding_model.encode(train["text"].tolist())
embedding_train

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/804 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

array([[ 0.03095995, -0.04980512, -0.00067632, ..., -0.04532453,
        -0.07391858, -0.00731021],
       [ 0.03705015, -0.01570939, -0.01769594, ...,  0.02685198,
         0.01320327, -0.06840479],
       [ 0.0294253 , -0.05950563,  0.01268852, ..., -0.04530559,
        -0.01561761, -0.02738026],
       ...,
       [-0.00462903, -0.05392354, -0.02290572, ..., -0.03905751,
        -0.03931314, -0.01579564],
       [ 0.01085776, -0.01724502, -0.01349181, ..., -0.07366002,
        -0.02297137,  0.01882051],
       [ 0.08405839, -0.0401996 ,  0.01693773, ..., -0.04509795,
        -0.04731593,  0.01547596]], dtype=float32)

In [18]:
train["text"]

0       enaku unmaikum aluha wantu thirunangaigal thei...
1       superstar vijay arivuketta polu ithu thappu pa...
2                                         ugka smile cute
3                                        anna waiting 🥰🥰🥰
4       yanda tamilnadu evvalavo pirachana athalam vit...
                              ...                        
5943                  noq day caste religious certificate
5944                                mimicry escape aitaru
5945                                           rajesh age
5946    videos nallarukku dont prank eena varavanga ep...
5947    avagaeaduthu kili kilinukilichitalumpoga nanba...
Name: text, Length: 5943, dtype: object

In [19]:
X_train = embedding_train

In [20]:
X_test = embedding_model.encode(test["text"].tolist())

In [21]:
y_train = train["category"].tolist()
y_test = test["category"].tolist()

In [22]:
from sklearn import svm
model = svm.SVC(kernel='linear', C=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
from sklearn.metrics import classification_report
#Summary of the predictions made by the classifier
print("SVM Algorithm")
print(classification_report(y_test, y_pred))

SVM Algorithm
              precision    recall  f1-score   support

           0       0.71      0.96      0.82       917
           1       0.60      0.15      0.24        40
           2       0.40      0.28      0.33        95
           3       0.65      0.41      0.51       218
           4       0.75      0.07      0.13        43
           5       0.00      0.00      0.00        53
           6       0.88      0.41      0.56        70
           7       0.00      0.00      0.00        50

    accuracy                           0.70      1486
   macro avg       0.50      0.29      0.32      1486
weighted avg       0.64      0.70      0.64      1486



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
model = svm.SVC(kernel='rbf', C=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

#Summary of the predictions made by the classifier
print("SVM Algorithm")
print(classification_report(y_test, y_pred))

SVM Algorithm
              precision    recall  f1-score   support

           0       0.72      0.97      0.83       917
           1       0.62      0.20      0.30        40
           2       0.39      0.29      0.34        95
           3       0.68      0.39      0.49       218
           4       1.00      0.16      0.28        43
           5       0.00      0.00      0.00        53
           6       0.94      0.43      0.59        70
           7       1.00      0.14      0.25        50

    accuracy                           0.71      1486
   macro avg       0.67      0.32      0.38      1486
weighted avg       0.69      0.71      0.66      1486



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_test

[0,
 2,
 0,
 6,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 7,
 0,
 0,
 3,
 6,
 0,
 0,
 2,
 0,
 2,
 7,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 2,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 4,
 0,
 2,
 3,
 2,
 5,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 4,
 0,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 7,
 5,
 0,
 0,
 0,
 0,
 0,
 7,
 2,
 1,
 0,
 6,
 0,
 2,
 0,
 0,
 0,
 3,
 3,
 4,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 0,
 3,
 0,
 7,
 3,
 1,
 3,
 3,
 0,
 0,
 0,
 5,
 1,
 2,
 5,
 3,
 2,
 0,
 0,
 0,
 5,
 0,
 2,
 0,
 0,
 0,
 3,
 4,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 4,
 0,
 0,
 7,
 7,
 4,
 3,
 0,
 4,
 7,
 0,
 3,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 2,
 0,
 0,
 0,
 0,
 6,
 0,
 3,
 0,
 7,
 2,
 0,
 3,
 2,
 0,
 3,
 6,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 3,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 5,
 0,
 0,
 2,
 5,
 0,
 2,
 2,
 3,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 2,
 0,
 6,
 4,
 0,
 6,
 7,
 0,
 3,
 2,
 0,
 0,


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train)
y_pred = mlp_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.84      0.80       917
           1       0.27      0.25      0.26        40
           2       0.39      0.33      0.36        95
           3       0.52      0.46      0.49       218
           4       0.32      0.26      0.29        43
           5       0.21      0.17      0.19        53
           6       0.61      0.51      0.56        70
           7       0.34      0.26      0.30        50

    accuracy                           0.66      1486
   macro avg       0.43      0.38      0.40      1486
weighted avg       0.64      0.66      0.65      1486



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [24]:
result = pd.read_csv('ta-en-misogyny-test.csv',sep='\t', names = ["text"])
result = result.dropna()
result

,text
0,ante peddi yedukkure ponnai vacci panam sambat...
1,Yella idangalilum pengalai asinga paduthuvathe...
2,Pannikku porandha paradhesi H Raja... Ponna pu...
3,Ayya white T-shirt! Tamil culture pathi mulusa...
4,Madu vandhi bjb fraudulent lady
...,...
1852,Avanga sonanga pari 8:50 sema
1853,Bro Avanga great bro semma character antha akk...
1854,NAGENDRAN NAGA hii Ur watsapp number
1855,Nengal solvathu sari endral angalum kuda vethi...


In [25]:
result['text']= result['text'].apply(lambda x:tolower(x))
result['text']= result['text'].apply(lambda x:removepunctuation(x))
result['text']= result['text'].apply(lambda x:deEmojify(x))
result['text']= result['text'].apply(lambda x:removestopwords(x))

In [26]:
Xf = embedding_model.encode(result["text"].tolist())

In [27]:
yf = model.predict(Xf)

In [28]:
yf.shape

(1857,)

In [29]:
key = { 0: 'None-of-the-above', 1:'Transphobic', 2: 'Counter-speech', 3:'Misandry',
       4:'Homophobia', 5:'Hope-Speech', 6:'Xenophobia', 7:'Misogyny'}

In [30]:
list_of_tuples = list(zip(result["text"].tolist(), yf.tolist()))

In [31]:
finalresult = pd.DataFrame(list_of_tuples,
                  columns = ['test_id', 'class_label'])

In [32]:
finalresult

,test_id,class_label
0,ante peddi yedukkure ponnai vacci panam sambat...,0
1,yella idangalilum pengalai asinga paduthuvathe...,0
2,pannikku porandha paradhesi raja ponna pundaiy...,3
3,ayya white tshirt tamil culture pathi mulusa p...,0
4,madu vandhi bjb fraudulent lady,0
...,...,...
1852,sonanga pari 850 sema,0
1853,great semma character antha akka avangalukku v...,0
1854,nagendran naga hii watsapp number,0
1855,nengal solvathu sari endral angalum kuda vethi...,2


In [33]:
finalresult["class_label"] = finalresult["class_label"].map(key)

In [34]:
finalresult

,test_id,class_label
0,ante peddi yedukkure ponnai vacci panam sambat...,None-of-the-above
1,yella idangalilum pengalai asinga paduthuvathe...,None-of-the-above
2,pannikku porandha paradhesi raja ponna pundaiy...,Misandry
3,ayya white tshirt tamil culture pathi mulusa p...,None-of-the-above
4,madu vandhi bjb fraudulent lady,None-of-the-above
...,...,...
1852,sonanga pari 850 sema,None-of-the-above
1853,great semma character antha akka avangalukku v...,None-of-the-above
1854,nagendran naga hii watsapp number,None-of-the-above
1855,nengal solvathu sari endral angalum kuda vethi...,Counter-speech


In [35]:
finalresult.to_csv("pandas_ta-en_run3.tsv", sep='\t', header = None)